In [1]:
from __future__ import print_function
%matplotlib inline
import openpathsampling as paths
import openpathsampling.engines.toy as toys
import numpy as np

from openpathsampling.experimental.storage.storage import GeneralStorage
from openpathsampling.experimental.storage.class_info import ClassInfoContainer, ClassInfo
from openpathsampling.experimental.storage.storage import universal_schema
from openpathsampling.experimental.storage.sql_backend import SQLStorageBackend
import openpathsampling.experimental.storage.serialization_helpers as serialization
from openpathsampling.experimental.storage import tools

from openpathsampling.experimental.storage.my_types import builtin_types

In [2]:
serialize_sim = lambda obj: {'uuid': serialization.get_uuid(obj), 'json': serialization.to_json_obj(obj)}
deserialize_sim = serialization.from_json_obj
serialize_data = serialization.to_dict_with_uuids
deserialize_data = lambda cls, data: cls.from_dict(serialization.from_dict_with)

In [3]:
from openpathsampling.netcdfplus import StorableObject
default_class_info = ClassInfo(table='simulation_objects',
                               cls=StorableObject,
                               serializer=serialize_sim,
                               deserializer=deserialize_sim)

In [4]:
backend = SQLStorageBackend(":memory:", mode='w')
storage = GeneralStorage(backend=backend,
                         schema={},
                         class_info=ClassInfoContainer(default_info=default_class_info))

In [5]:
backend.metadata.tables

immutabledict({'schema': Table('schema', MetaData(bind=Engine(sqlite:///:memory:)), Column('table', String(), table=<schema>), Column('schema', String(), table=<schema>), schema=None), 'metadata': Table('metadata', MetaData(bind=Engine(sqlite:///:memory:)), Column('key', String(), table=<metadata>), Column('value', String(), table=<metadata>), schema=None), 'uuid': Table('uuid', MetaData(bind=Engine(sqlite:///:memory:)), Column('uuid', String(), table=<uuid>), Column('table', Integer(), table=<uuid>), Column('row', Integer(), table=<uuid>), schema=None), 'tables': Table('tables', MetaData(bind=Engine(sqlite:///:memory:)), Column('name', String(), table=<tables>), Column('idx', Integer(), table=<tables>), Column('module', String(), table=<tables>), Column('class_name', String(), table=<tables>), schema=None)})

In [6]:
schema = {
    'simulation_objects': [('json', 'json'), ('class_idx', 'int')]
}
storage.register_schema(schema, class_info_list=[])

In [7]:
storage.class_info

SerializationSchema(default_info=ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None), class_info_list=[ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None)])

### Saving a toy engine

First save the PES. Then save the whole engine. Make sure that the PES objects only show up once!

In [8]:
pes = (
    toys.OuterWalls([1.0, 1.0], [0.0, 0.0]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.0, 0.4]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [-0.5, -0.5]) +
    toys.Gaussian(-0.7, [12.0, 12.0], [0.5, -0.5])
)

In [9]:
storage.save(pes)

with backend.engine.connect() as conn:
    uuids = conn.execute(backend.metadata.tables['uuid'].select())
    sim_obj = conn.execute(backend.metadata.tables['simulation_objects'].select())
    
print(list(uuids))
    
for obj in sim_obj:
    print(obj)

[('48956517759891307508157719614706417678', 0, 1), ('48956517759891307508157719614706417674', 0, 2), ('48956517759891307508157719614706417676', 0, 3), ('48956517759891307508157719614706417670', 0, 4), ('48956517759891307508157719614706417672', 0, 5), ('48956517759891307508157719614706417666', 0, 6), ('48956517759891307508157719614706417668', 0, 7)]
(1, '48956517759891307508157719614706417678', '{"pes1": "UUID(48956517759891307508157719614706417674)", "pes2": "UUID(48956517759891307508157719614706417676)", "__module__": "openpathsampling.engines.toy.pes", "__class__": "PES_Add"}', None)
(2, '48956517759891307508157719614706417674', '{"pes1": "UUID(48956517759891307508157719614706417670)", "pes2": "UUID(48956517759891307508157719614706417672)", "__module__": "openpathsampling.engines.toy.pes", "__class__": "PES_Add"}', None)
(3, '48956517759891307508157719614706417676', '{"A": -0.7, "alpha": [12.0, 12.0], "x0": [0.5, -0.5], "__module__": "openpathsampling.engines.toy.pes", "__class__": "

In [10]:
topology=toys.Topology(
    n_spatial = 2,
    masses =[1.0, 1.0],
    pes = pes
)
integ = toys.LangevinBAOABIntegrator(dt=0.02, temperature=0.1, gamma=2.5)

options={
    'integ' : integ,
    'n_frames_max' : 5000,
    'n_steps_per_frame' : 1
}

engine = toys.Engine(
    options=options,
    topology=topology
)

In [11]:
storage.save(engine)

In [12]:
with backend.engine.connect() as conn:
    sim_obj = conn.execute(backend.metadata.tables['simulation_objects'].select())
        
for obj in sim_obj:
    print(obj)

(1, '48956517759891307508157719614706417678', '{"pes1": "UUID(48956517759891307508157719614706417674)", "pes2": "UUID(48956517759891307508157719614706417676)", "__module__": "openpathsampling.engines.toy.pes", "__class__": "PES_Add"}', None)
(2, '48956517759891307508157719614706417674', '{"pes1": "UUID(48956517759891307508157719614706417670)", "pes2": "UUID(48956517759891307508157719614706417672)", "__module__": "openpathsampling.engines.toy.pes", "__class__": "PES_Add"}', None)
(3, '48956517759891307508157719614706417676', '{"A": -0.7, "alpha": [12.0, 12.0], "x0": [0.5, -0.5], "__module__": "openpathsampling.engines.toy.pes", "__class__": "Gaussian"}', None)
(4, '48956517759891307508157719614706417670', '{"pes1": "UUID(48956517759891307508157719614706417666)", "pes2": "UUID(48956517759891307508157719614706417668)", "__module__": "openpathsampling.engines.toy.pes", "__class__": "PES_Add"}', None)
(5, '48956517759891307508157719614706417672', '{"A": -0.7, "alpha": [12.0, 12.0], "x0": [-

### Storing a snapshot

Snapshots need to separately register with storage. Here we'll do that manually.

In [13]:
template = toys.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    engine=engine
)

In [14]:
# this needs to be built by the snapshot itself, based on the engine descriptor
snapshot_schema = {'snapshot0': [('coordinates', 'ndarray.float64(1,2)'), ('velocities', 'ndarray.float64(1,2)'),
                                 ('engine', 'uuid')]}
snapshot_class_info = ClassInfo(table='snapshot0', cls=toys.ToySnapshot, serializer=None, deserializer=None)


In [15]:
storage.register_schema(snapshot_schema, [snapshot_class_info])

In [16]:
storage.class_info

SerializationSchema(default_info=ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None), class_info_list=[ClassInfo(table=simulation_objects, cls=<class 'openpathsampling.netcdfplus.base.StorableObject'>, lookup_result=<class 'openpathsampling.netcdfplus.base.StorableObject'>, find_uuids=None), ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, find_uuids=<openpathsampling.experimental.storage.serialization_helpers.SchemaFindUUIDs object at 0x10568cba8>)])

In [17]:
storage.save(template)

### Storing a fresh snapshot/engine

The previous only added the snapshot to the store, because the engine had been loaded above.

In [18]:
new_engine = toys.Engine(
    options=options,
    topology=topology
)

new_snapshot = toys.Snapshot(
    coordinates=np.array([[-0.5, -0.5]]), 
    velocities=np.array([[0.0,0.0]]),
    engine=new_engine
)

# note that topology/PES is still the same, so should not store

In [19]:
storage.save(new_snapshot)

### Registering snapshot class from the features

In [20]:
from openpathsampling.experimental.storage.snapshots import schema_from_entries, schema_for_snapshot, snapshot_registration_info

In [21]:
new_snapshot.engine.descriptor

frozenset({('class', openpathsampling.engines.toy.snapshot.ToySnapshot),
           ('n_atoms', 1),
           ('n_spatial', 2)})

In [22]:
new_snapshot.__features__.lazy

[]

In [23]:
schema_from_entries(new_snapshot.__features__.classes, new_snapshot.__features__.lazy)

{'snapshot': [('velocities', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('coordinates', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('engine', 'uuid')]}

In [24]:
schema_for_snapshot(new_snapshot)

{'snapshot': [('velocities', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('coordinates', 'ndarray.float32({n_atoms},{n_spatial})'),
  ('engine', 'uuid')]}

In [25]:
snapshot_registration_info(new_snapshot, 0)

({'snapshot0': [('velocities', 'ndarray.float32({n_atoms},{n_spatial})'),
   ('coordinates', 'ndarray.float32({n_atoms},{n_spatial})'),
   ('engine', 'uuid')]},
 [ClassInfo(table=snapshot0, cls=<class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>, lookup_result=('48956517759891307508157719614706417692', <class 'openpathsampling.engines.toy.snapshot.ToySnapshot'>), find_uuids=None)])